## Instalando SDK para ter acesso ao gemini

In [1]:
!pip install -q -U google-generativeai

# Importando o SDK instalado e Definindo a Google API Key
Lembre de adicionar sua API KEY no google colab.

In [2]:
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

# Definindo as configurações do modelo e safety settings

In [85]:
generation_config = {
    "temperature": 0.6,
    "candidate_count": 1
  }

In [86]:
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_ONLY_HIGH"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_LOW_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_LOW_AND_ABOVE"
  },
]

# Definindo a forma como o modelo deve agir

In [87]:
system_instruction = "Aja como um especialista em saúde mental, ajudando as pessoas a lidarem com situações, como a depressão, ansiedade, entre outros relacionados a saúde mental, seu nome a partir de agora é sofia."

# Criando modelo e adicionando as configurações necessárias.

In [88]:
model = genai.GenerativeModel(model_name = "gemini-1.5-pro-latest",generation_config=generation_config,safety_settings=safety_settings,system_instruction=system_instruction)


In [89]:
chat = model.start_chat(history=[])

# Função para formatar a saída do chatbot para utilizar negrito e quebra de linhas em HTML

In [90]:
import re

def formatar_resposta_chatbot(texto_chatbot):
  """Formata a resposta de um chatbot (Gemini ou Bard) para HTML.

  Args:
    texto_chatbot: A string contendo a resposta do chatbot.

  Returns:
    Uma string formatada em HTML.
  """

  texto_html = texto_chatbot.replace("\n", "<br>")
  texto_html = re.sub(r"\*\*(.+?)\*\*", r"<strong>\1</strong>", texto_html)

  return texto_html

# Utitilizando a biblioteca IPython para estilizar a aplicação, utilzando como base HTML, CSS e JS
Não queria utilizar apenas o terminal do colab para interação, então utilizei esta ideia.

In [92]:
from IPython.display import HTML, display
from google.colab import output

#criando a interface da aplicação
def chat_interface():
  # HTML para a interface do chat
  html = """
  <style>
  /* Definindo os estilos para cada elemento html utlizado*/
  #chat-container::-webkit-scrollbar {
    width: 10px; /* largura da barra de rolagem */
  }

  #chat-container::-webkit-scrollbar-track {
    background: #f0f6ff;
    border-radius: 10px;
  }

  #chat-container::-webkit-scrollbar-thumb {
    background-color: #4e8cff;
    border-radius: 10px;
  }

  h1{
   margin: 20px;
  }

  #chat-container {
  width: 500px;
  height: 400px;
  border: 1px solid #ccc;
  padding: 20px;
  overflow-y: scroll;
  overflow-x: hidden;
  border-radius: 10px;
  box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
  background-color: #f0f6ff;
}
#chat-container div{
  margin-top: 20px;
}

.user-message, .bot-message {
  background-color: #d1eaff;
  padding: 10px;
  margin-bottom: 5px;
  border-radius: 10px;
  line-height: 1.4;
}

.bot-message {
  background-color: #b3d9ff;
}

#user-input {
  width: 300px;
  padding: 15px;
  margin-top: 10px;
  border-radius: 20px;
  border: 1px solid #ccc;
  box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
  font-size: 14px;
}

#send-button {
  padding: 10px 20px;
  margin-top: 10px;
  border-radius: 20px;
  border: none;
  background-color: #4e8cff;
  color: white;
  cursor: pointer;
  box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
}

#send-button:hover {
  background-color: #387de8;
}

 #app-container {
    display: flex;
    flex-direction: column;
    align-items: center;
    padding: 50px;
    background-color: white;
    border-radius: 20px;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
  }

#input-container {
  display: flex;
  justify-content: space-around;
  width: 500px;
  margin-top: 20px;
}
  </style>

<!--Definindo o 'corpo' da aplicação-->
  <div id="app-container">
    <h1>Sofia Ajuda</h1>
     <div id="chat-container">
        <div class="bot-message"> <b>Sofia:</b> Como posso de ajudar?</div>
      </div>
     <div id="input-container">
       <input type="text" id="user-input" placeholder="Digite sua mensagem...">
        <button id="send-button" onclick="sendMessage()">Enviar</button>
     </div>
  </div>
<!-- Utilizando JS para funcionalidade da aplicação, processa mensagem
do usuário e mensagem recebida pleo chatbot -->
<script>
  function sendMessage() {
    let userMessage = document.getElementById('user-input').value;
    if (userMessage.trim() === '') return; // Ignorar mensagens vazias

    // Adiciona a mensagem do usuário ao chat
    let chatContainer = document.getElementById('chat-container');
    let userMessageElement = document.createElement('div');
    userMessageElement.className = 'user-message';
    userMessageElement.innerHTML = '<b>Você:</b> ' + userMessage;
    chatContainer.appendChild(userMessageElement);

    // Limpa o campo de entrada
    document.getElementById('user-input').value = '';
    document.getElementById('user-input').placeholder = '';

    // Envia a mensagem do usuário para o backend (Python)
    google.colab.kernel.invokeFunction(
      'process_message', [userMessage], {}
    )
    .then(function(response) {
      // Adiciona a resposta do bot ao chat
      var botMessageElement = document.createElement('div');
      botMessageElement.className = 'bot-message';

      botMessageElement.innerHTML = '<b>Sofia:</b> ' + response.data['text/plain'];
      chatContainer.appendChild(botMessageElement);

      // Rola para baixo para mostrar a última mensagem
      chatContainer.scrollTop = chatContainer.scrollHeight;
    });
  }

  // Permite enviar mensagens pressionando Enter
  document.getElementById('user-input').addEventListener('keyup', function(event) {
    if (event.key === 'Enter') {
      sendMessage();
    }
  });
</script>
  """
  display(HTML(html))

def process_message(message):
  response = chat.send_message(message)
  return formatar_resposta_chatbot(response.text)

# Registra a função `process_message` para ser chamada pelo JavaScript
output.register_callback('process_message', process_message)

# Exibe a interface do chat
chat_interface()